<a href="https://colab.research.google.com/github/cagBRT/promptEngineering/blob/main/OPENAI_DATA_MARKETING_LABS_OpenAI_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DATA MARKETING LABS : Text Generation

- FR : https://www.datamarketinglabs.com/guide-complet-sur-gpt-3-part2

- EN : https://www.datamarketinglabs.org/gpt-3-a-full-guide-technical-part

Objectives : Use Fine-Tuning with GPT-3


## Step 1 : Install OpenAI

In [1]:
!pip install --upgrade openai

import os
import openai
openai.organization = ""
openai.api_key = ""
#openai.Engine.list()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 7.1 MB/s eta 0:00:00


## Step 2 : Import Libraries

In [2]:
import re
import json
import pandas as pd
import openai
import string
import requests
import random

## Step 3 : CLI data preparation tool

Here, my file : data-tomato.json


In [3]:
{"prompt":"Tomato Description\n\n nom:Tomate Black mountain pink op\n\n	port:indéterminé\n\ncalibre:très gros fruit - masse 300g à 1kg - calibre supérieur à 85mm\n\n	origine:États-Unis\n\nhauteur:comprise entre 1.2 et 2 mètres\n\nusage:apte à la cuisson - coulis - sauce\n\ncouleur:rose\n\nprecocite:Tardive - 80 à 100 jours\n\n
description:", "completion":"La Tomate Black mountain pink op vient des États-Unis. Que vous vouliez préparer un coulis, des sauces ou tout autre plat qui nécessite une cuisson, alors cette variété est adaptée à vos besoins. C'est une tomate rose, à port indéterminé. Elle produit de très gros fruits, pouvant faire de 300g à 1kg. Le calibre des tomates peut être supérieur à 85mm. C'est une tomate tardive, qui met entre 80 et 100 jours pour arriver à maturité. Le plant de la tomate Black mountain pink op peut aller de 1.2m à 2 mètres."}
{"prompt":"Tomato description\n\nnom:Tomate Amethyst Jewel\n\nport:indéterminé\n\ncalibre:petit fruit - masse 50 à 100g - calibre 47 à 57mm\n\norigine:États-Unis\n\nhauteur:comprise entre 1.2 et 2 mètres\n\nusage:salade\n\nproduction:très bonne\n\ncouleur:bigarrée-zébrée\n\nprecocite:Mi saison - 65 à 80 jours\n\n
description:", "completion":"La tomate Amethyst Jewel est une tomate bigarrée-zébrée qui est très bonne en salade. Ses fruits sont petits, avec un calibre de 47 à 57mm, pour un poids allant de 50 à 100g. Originaire des États-Unis, la tomate Amethyst Jewel est une variété à port indéterminé, dont la taille est comprise entre 1.2 et 2 mètres. C'est une variété de mi-saison. Vous la récolterez entre 65 et 80 jours après repiquage."}
{"prompt":"Tomato description\n\nnom:Tomate Adelin Morin\n\nport:indéterminé\n\ncalibre:très gros fruit - masse 300g à 1kg - calibre supérieur à 85mm\n\norigine:Canada\n\nhauteur:comprise entre 1.2 et 2 mètres\n\nusage:salade\n\nproduction:très bonne\n\ncouleur:rose\n\nmaladie:assez bonne résistance aux maladies - oïdium - mildiou\n\nprecocite:Mi saison - 65 à 80 jours\n\ndescription:", "completion":"La variété de tomate Adelin Morin est résistante aux maladies comme le mildiou et l'oïdium. En plus d'être résistante, c'est une tomate au gout excellent, à cuisiner en salade. Originaire du Canada, c'est une variété de tomate à port indéterminé, qui produit des tomates roses. La tomate Adelin Morin est un très gros fruit pouvant aller de 300g à 1kg. Cette tomate, qui se récolte à la mi-saison, pousse sur un plant qui peut mesurer jusqu'à 2 mètres de hauteur."}
{"prompt":"Tomato description\n\nnom:Tomate belarus orange\n\nport:déterminé\n\ncalibre:fruit moyen - masse 100 à 150g - calibre 57 à 67mm\n\norigine:Bélarus\n\n	hauteur:comprise entre 50 et 80 cm\n\n
usage:longue conservation - aptitude au séchage\n\nproduction:très bonne\n\ncouleur:orange\n\nprecocite:Mi saison - 65 à 80 jours\n\ndescription:", "completion":"Si vous recherchez une tomate savoureuse et orange, alors la tomate belarus orange pourrait vous intéresser. Elle est parfaitement adaptée pour la longue conservation ainsi qu'au séchage. Issue de Bélarus, c'est une variété à port déterminé qui produit des fruits de taille moyenne, avec un poids compris entre 100 et 150 grammes. Le plant se développe sur une hauteur comprise entre 50 et 80cm et produit des fruits allant de 57 à 67mm de calibre. La tomate belarus orange est de mi-saison. Elle met entre 65 et 80 jours pour que les fruits commencent à arriver à maturité."}
{"prompt":"Tomato description\n\nnom:Tomate Amana Orange Op\n\nport:indéterminé\n\ncalibre:très gros fruit - masse 300g à 1kg - calibre supérieur à 85mm\n\norigine:États-Unis\n\n
hauteur:comprise entre 1.2 et 2 mètres\n\nusage:apte à la cuisson - coulis - sauce\n\nproduction:très bonne\n\ncouleur:orange\n\nprecocite:Mi saison - 65 à 80 jours\n\ndescription:", "completion":"Vous recherchez des tomates pour créer des coulis, des sauces ? La tomate Amana Orange Op est apte à la cuisson. De plus, ses gros fruits, pouvant aller jusqu'à 1kg, vous permettront de créer vos sauces et coulis en quantité. D'un calibre supérieur à 85mm, cette variété à port indéterminé, issue des États-Unis est très goûteuse. Arrivée à maturité à la mi-saison (entre 65 et 80 jours), La tomate Amana Orange Op vous donnera des fruits orange, sur un plant qui peut aller jusqu'à 2 mètres de hauteur."}
{"prompt":"Tomato description\n\nnom:Tomate Basket vee op\n\nport:indéterminé\n\ncalibre:gros fruit - masse 150 à 300g - calibre 67 à 85mm\n\n
origine:Canada\n\n	couleur:orange\n\nhauteur:comprise entre 50 et 80 cm\n\nusage: apte à la cuisson - coulis - sauce\n\ncouleur:rouge\n\nmaladie:assez bonne résistance aux maladies - oïdium - mildiou\n\nprecocite:Précoce - 42 à 55 jours\n\ndescription:", "completion":"La variété de Tomate Basket vee op, issu du Canada, cumule les avantages : - c'est une tomate précoce (récolte en 42 à 55 jours après le repiquage), tout en ayant de gros fruits (150 à 300g pour un calibre de 67 à 85mm) - elle est résistante aux maladies (l'oïdium et le mildiou) La Tomate Basket vee op est à port indéterminé, mais elle ne dépasse pas les 80 cm de hauteur. Elle produit des tomates rouges. Elle est bonne, même après la cuisson. Ce qui en fait une tomate adaptée aux sauces et coulis."}
{"prompt":"Tomato description\n\nnom:Tomate Belmonte op\n\nport:indéterminé\n\ncalibre:très gros fruit - masse 300g à 1kg - calibre supérieur à 85mm\n\norigine:Italie\n\nhauteur:comprise entre 1.2 et 2 mètres\n\nusage:apte à la cuisson - coulis - sauce\n\ncouleur:rose\n\nprecocite:Mi saison - 65 à 80 jours\n\ndescription:", "completion":"La variété Belmonte op a vu le jour en Italie. Rose, elle est propice à la conception de sauces et de coulis. En effet, c'est une tomate qui ne perd pas sa saveur lors de la cuisson. Vous pourrez la récolter après 65 jours minimum après le repiquage, et 80 jours maximum. C'est une tomate à port indéterminée (qui peut mesurer de 1.2m à 2m) qui produit de très gros fruits. D'un calibre de 85mm en moyenne, ils peuvent peser de 300 à 1000 grammes."}
{"prompt":"Tomato description\n\nnom:Tomate Aunt Ruby's German Green Cherry op\n\nport:indéterminé\n\ncalibre:cocktail - masse de 30 à 50g\n\norigine:États-Unis\n\n
hauteur:comprise entre 1.2 et 2 mètres\n\nusage:salade\n\nproduction:bonne\n\ncouleur:verte\n\nprecocite:Mi saison de 65 à 80 jours\n\ndescription:", "completion":"La variété de Tomate Aunt Ruby's German Green Cherry op est une tomate cocktail, c'est-à-dire qu'elle est très sucrée. Originaire des États-Unis, elle est excellente pour donner un goût exceptionnel à vos sales. Elle est originale car elle est verte et petite. En effet, elle pèse entre 30 et 50g maximum. La variété de Tomate Aunt Ruby's German Green Cherry op est à port indéterminé, qui se récolte entre 65 et 80 jours après plantation."}
{"prompt":"Tomato description\n\nnom:Tomate Bezrazmernyi\n\nport:indéterminé\n\ncalibre:très gros fruit - masse 300g à 1kg - calibre supérieur à 85mm\n\norigine:Russie\n\nhauteur:comprise entre 1.2 et 2 mètres\n\nusage:apte à la cuisson - coulis - sauce\n\nproduction:bonne\n\ncouleur:rouge\n\nmaladie:assez bonne résistance aux maladies - oïdium - mildiou\n\nprecocite:Mi saison de 65 à 80 jours\n\ndescription:", "completion":"La tomate Bezrazmernyi est une tomate indéterminée qui est assez résistante aux maladies comme le mildiou et l'oïdium. De mi-saison, elle est adaptée pour les coulis et les sauces. Elle a un assez bon goût, et produit de très gros fruits allant de 300g à 1kg, avec une circonférence dépassant les 85mm. La tomate Bezrazmernyi est originaire de la Russie."}


SyntaxError: ignored

In [4]:
!openai tools fine_tunes.prepare_data -f data-tomato.json

Analyzing...


ERROR in read_any_format validator: File data-tomato.json does not exist.

Aborting...

## Step 4 : Create a fine-tuned model

In [ ]:
%env OPENAI_API_KEY=sk-wwXXXXXXXXXXXXXXXXXXXXXXXXXX

!openai api fine_tunes.create -t "data-tomato_prepared.jsonl" -m "curie"

When the job is done, it should display the name of the fine-tuned model.

In addition to creating a fine-tune job, you can also list existing jobs, retrieve the status of a job, or cancel a job.

In [ ]:
# List all created fine-tunes
!openai api fine_tunes.list

## Step 5 : Use a fine-tuned model

In [ ]:
YOUR_MODEL = "curie:ft-datamarketinglabs-2021-07-22-12-03-32"
YOUR_PROMPT = "Tomato description\n\nnom:Tomate Terrasi\n\ncalibre:très gros fruit - masse 300g à 1kg - calibre supérieur à 85mm\n\norigine:Russie\n\nhauteur:comprise entre 1.2 et 2 mètres\n\nusage:apte à la cuisson - coulis - sauce\n\nproduction:bonne\n\ncouleur:rouge\n\nmaladie:assez bonne résistance aux maladies - oïdium - mildiou\n\nprecocite:Mi saison de 65 à 80 jours\n\ndescription:"

Quick test without parameters

In [ ]:
!openai api completions.create -m YOUR_MODEL -p YOUR_PROMPT

## Step 6 : Write Python Function

In [ ]:
def completion(prompt, engine_id, debug=True, **kwargs):

    COMPLETION_ENDPOINT = 'https://api.openai.com/v1/completions'
    
    headers = {'Authorization': 'Bearer {api_key}'.format(api_key=openai.api_key),
              'Content-Type': 'application/json'}

    data = {
              "prompt": prompt,
              "model": engine_id,
              "max_tokens": kwargs.get('max_tokens', 200),
              "temperature": kwargs.get('temperature', 0.3),
              "frequency_penalty": kwargs.get('frequency_penalty', 0.9),            
              "stop": [""]
            }
    
    data.update(kwargs)

    response = requests.post(COMPLETION_ENDPOINT, headers=headers, data=json.dumps(data))
    result = response.json()

    if debug:
        print('Headers:')
        print(json.dumps(headers, indent=4))
        print('Data:')
        print(json.dumps(data, indent=4))
        print('Result:')
        print(json.dumps(result, indent=4))

    if response.status_code == 200:
        return [x['text'].strip() for x in result['choices']]
    else:
        return "Error: {}".format(result['error']['message'])
  
completion(YOUR_PROMPT, YOUR_MODEL, debug=False)

The result is impressive with only 9 examples. In general, OpenAI recommends having at least a few hundred examples. They found that performance tends to linearly increase for every doubling of the number of examples.

In [ ]:
YOUR_PROMPT = "Tomato description\n\nnom:Tomate Babbardi\n\ncalibre:petit fruit - masse 30g à 100g - calibre inférieur à 45mm\n\norigine:France\n\nhauteur:comprise entre 1.2 et 1.5 mètres\n\nusage:purée\n\nproduction:bonne\n\ncouleur:rouge\n\nmaladie:assez bonne résistance aux maladies - covidou\n\nprecocite:Mi saison de 5 à 15 jours\n\ndescription:"

completion(YOUR_PROMPT, YOUR_MODEL, debug=False)